In [9]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

In [10]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [11]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label

print(np.max(label_visiprog))

202.0


In [12]:
accuracy = []
accuracy_vsp = []

N_material = 61
N_material_train = 10

for fold in range(10):
    try:
        print("Trial {}".format(fold))

        idx_train = np.array([])
        for i in range(N_material_train):
            train_class = np.random.randint(0, N_material)
            idx_train= np.concatenate((idx_train, np.where(label_material == train_class)[0]))

        # need to get better code later
        idx_test = np.setdiff1d(np.arange(N), idx_train)

        idx_train = idx_train.astype(int)
        idx_test = idx_test.astype(int)

        X_train = raw_feature[idx_train,:]
        Y_VSP_train = label_visiprog[idx_train]
        Y_mat_train = label_material[idx_train]

        X_test = raw_feature[idx_test,:]
        Y_VSP_test = label_visiprog[idx_test]
        Y_mat_test = label_material[idx_test]

        model = LFDA()
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy.append(res['accuracy'])
        
        model_vsp = LFDA_VISIPROG()
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])

    except:
        print("Skipping trial {}".format(fold))

Trial 0
Trial 1
Trial 2
Trial 3
Trial 4
Trial 5
Trial 6
Trial 7
Trial 8
Trial 9


In [13]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy})
display(df)
display(df.mean())

,material,visiprog
0,0.982343,0.979133
1,0.975383,0.966068
2,0.958527,0.948769
3,0.957203,0.924774
4,0.977257,0.965098
5,0.974235,0.964803
6,0.962011,0.956083
7,0.964656,0.963980
8,0.984845,0.975797
9,0.969130,0.964398


material    0.970559
visiprog    0.960890
dtype: float64

In [ ]:
print(model.transformer())